Receiving teams and players into la_liga_players.csv

In [ ]:
import os
import requests
import pandas as pd
import time
import psycopg2
from datetime import datetime

API_KEY = "use_your_own_api_key"
def get_la_liga_players_with_details(api_key):
    headers = {'X-Auth-Token': api_key}
    base_url = "https://api.football-data.org/v4"
    
    competitions_url = f"{base_url}/competitions/PD/teams"
    
    try:
        response = requests.get(competitions_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при получении команд: {e}")
        return None
    
    teams = response.json()['teams']
    print(f"Найдено команд: {len(teams)}")
    
    all_players = []
    
    for i, team in enumerate(teams, 1):
        print(f"[{i}/{len(teams)}] Обрабатываем команду: {team['name']}")
        
        # Получаем детальную информацию о команде (включая состав)
        team_url = f"{base_url}/teams/{team['id']}"
        
        try:
            team_response = requests.get(team_url, headers=headers)
            team_response.raise_for_status()
            team_data = team_response.json()
            
            # Обрабатываем игроков
            for player in team_data.get('squad', []):
                player_info = {
                    'api_id': player.get('id'),
                    'name': player.get('name'),
                    'position': player.get('position', 'Unknown'),
                    'nationality': player.get('nationality', 'Unknown'),
                    'team_id': team['id'],
                    'team_name': team['name'],
                    'team_short_name': team.get('shortName', ''),
                    'team_crest': team.get('crest', ''),
                    'date_created': datetime.now()
                }
                all_players.append(player_info)
            
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при получении данных команды {team['name']}: {e}")
            continue
        
        # Задержка для соблюдения лимитов API 
        time.sleep(6)
    
    return pd.DataFrame(all_players)

# Исполняемая часть
if __name__ == "__main__":
    df_players = get_la_liga_players_with_details(API_KEY)
    
    if df_players is not None:
        print(f"\nУспешно получены данные о {len(df_players)} игроках")
        print("\nПервые 5 записей:")
        print(df_players.head())
        
        # Сохраняем в CSV для проверки
        df_players.to_csv('la_liga_players.csv', index=False, encoding='utf-8')
        print("\nДанные сохранены в 'la_liga_players.csv'")
        
    else:
        print("Не удалось получить данные")
        

Parcing teams and players into insert_data.sql

In [ ]:
import pandas as pd

class SimpleLaLigaParser:
    def __init__(self, csv_file_path):
        self.csv_file_path = csv_file_path
        
    def parse_data(self):
        try:
            df = pd.read_csv(self.csv_file_path)
            print(f"Загружено {len(df)} записей из CSV файла")
            
            teams_data = self._get_teams_data(df)
            players_data = self._get_players_data(df)
            
            return {
                'teams': teams_data,
                'players': players_data
            }
            
        except Exception as e:
            print(f"Ошибка: {e}")
            return None
    
    def _get_teams_data(self, df):
        teams = df[['team_id', 'team_name', 'team_short_name']].drop_duplicates()
        
        teams_list = []
        for _, team in teams.iterrows():
            team_info = {
                'team_id': int(team['team_id']),
                'team_name': team['team_name'],
                'team_short_name': team['team_short_name']
            }
            teams_list.append(team_info)
        
        print(f"Найдено {len(teams_list)} команд")
        return teams_list
    
    def _get_players_data(self, df):
        players_list = []
        
        for index, row in df.iterrows():
            player_info = {
                'player_id': index + 1,
                'api_id': int(row['api_id']),
                'name': row['name'].replace("'", "''"),  # Экранирование кавычек
                'position': row['position'],
                'nationality': row['nationality'],
                'team_id': int(row['team_id'])
            }
            players_list.append(player_info)
        
        print(f"Найдено {len(players_list)} игроков")
        return players_list
    
    def generate_sql(self):
        data = self.parse_data()
        
        if not data:
            return None
        
        # SQL для teams
        teams_sql = "INSERT INTO teams (team_id, team_name, team_short_name) VALUES\n"
        teams_values = []
        for team in data['teams']:
            values = f"({team['team_id']}, '{team['team_name']}', '{team['team_short_name']}')"
            teams_values.append(values)
        teams_sql += ",\n".join(teams_values) + ";\n"
        
        # SQL для players
        players_sql = "INSERT INTO players (player_id, api_id, name, position, nationality, team_id) VALUES\n"
        players_values = []
        for player in data['players']:
            values = f"({player['player_id']}, {player['api_id']}, '{player['name']}', '{player['position']}', '{player['nationality']}', {player['team_id']})"
            players_values.append(values)
        players_sql += ",\n".join(players_values) + ";\n"
        
        return {
            'teams': teams_sql,
            'players': players_sql
        }
    
    def save_sql_to_file(self, filename="insert_data.sql"):
        sql_scripts = self.generate_sql()
        
        if not sql_scripts:
            print("Не удалось сгенерировать SQL скрипты")
            return
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("-- SQL команды для заполнения таблиц teams и players\n\n")
            f.write(sql_scripts['teams'])
            f.write("\n")
            f.write(sql_scripts['players'])
        
        print(f"SQL команды сохранены в файл: {filename}")

csv_file = "la_liga_players.csv"
parser = SimpleLaLigaParser(csv_file)
parser.save_sql_to_file()

Receiving data from API Football-Data.org for inserting and parcing stadiums into stadiums_data

In [ ]:
import requests
import json
#Football-Data.org
def get_la_liga_stadiums_from_api():

    # API ключ
    API_KEY = "use_your_own_api_key"
    
    headers = {'X-Auth-Token': API_KEY}
    url = "https://api.football-data.org/v4/competitions/PD/teams"
    
    try:
        print("Запрос данных с Football-Data.org...")
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            teams = data['teams']
            
            stadiums_data = []
            stadium_id = 1
            
            for team in teams:
                if 'venue' in team and team['venue']:
                    stadium_info = {
                        'stadium_id': stadium_id,
                        'name': team['venue'],
                        'city': team.get('address', 'Unknown'),
                        'team_name': team['name'],
                        'team_id': team['id']
                    }
                    stadiums_data.append(stadium_info)
                    stadium_id += 1
            
            print(f"Получено данных о {len(stadiums_data)} стадионах")
            return stadiums_data
        else:
            print(f"Ошибка API: {response.status_code}")
            print(f"Ответ: {response.text}")
            return None
            
    except Exception as e:
        print(f"Ошибка при запросе: {e}")
        return None

def save_stadiums_to_sql_file(stadiums_data, filename="insert_stadiums.sql"):
    if not stadiums_data:
        print("Нет данных для сохранения")
        return
    
    sql_content = """
-- Источник: Football-Data.org

INSERT INTO stadiums (stadium_id, name, city, team_id) VALUES
"""
    
    values = []
    for stadium in stadiums_data:
        # Экранирование кавычек в названиях
        name = stadium['name'].replace("'", "''")
        city = stadium['city'].replace("'", "''")
        value = f"({stadium['stadium_id']}, '{name}', '{city}', {stadium['team_id']})"
        values.append(value)
    
    sql_content += ",\n".join(values) + ";\n"
    
    # Сохраняем в файл
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(sql_content)
    
    print(f"Данные сохранены в файл: {filename}")

def main():
    # Получаем данные с API
    stadiums_data = get_la_liga_stadiums_from_api()
    
    if stadiums_data:
        # Сохраняем в SQL файл
        save_stadiums_to_sql_file(stadiums_data)

        for stadium in stadiums_data:
            print(f"{stadium['name']} | {stadium['city']} | {stadium['team_name']}")
        
        print(f"\nВсего обработано: {len(stadiums_data)} стадионов")
    else:
        print("Не удалось получить данные с API")

if __name__ == "__main__":
    main()